In [282]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nicolehoelzl/mpii-human-pose-data")

print("Path to dataset files:", path)

C:\Users\vishw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.47M/1.47M [00:01<00:00, 1.07MB/s]

Extracting files...
Path to dataset files: C:\Users\vishw\.cache\kagglehub\datasets\nicolehoelzl\mpii-human-pose-data\versions\1
